In [1]:
import pandas as pd
import numpy as np
import gzip
import json

To parse data from json and info

In [2]:

def parse_data(info_path, json_zip_path):
   # loads data
   print(f"Loading {json_zip_path}...")
   with gzip.open(json_zip_path, "r") as f:
      data = [json.loads(line) for line in f]
   
   # loads data with label
   print(f"Loading {info_path}...")
   info = pd.read_csv(info_path)

   #transfer information from json dict to list
   print("Transferring data from json to dataframe...")
   res = []
   for row in data:
      for trans_id in row.keys():
         for trans_pos in row[trans_id].keys():
            for nucleo_seq in row[trans_id][trans_pos].keys():
               temp = list(np.mean(np.array(row[trans_id][trans_pos][nucleo_seq]), axis=0))
               # to get raw data without aggregation
               # for features in row[trans_id][trans_pos][nucleo_seq]:
               res.append([trans_id, int(trans_pos), nucleo_seq] + temp)

   data = pd.DataFrame(res, columns = ['transcript_id', 'transcript_pos', 'nucleo_seq',
                                       'dwell_time_-1', 'sd_-1', 'mean_-1',
                                       'dwell_time_0', 'sd_0', 'mean_0',
                                       'dwell_time_1', 'sd_1', 'mean_1'
                                       ])
   # Merge json data with labels
   print("Merging dataframes to obtain labels")
   merged_data = pd.merge(data,info,left_on=["transcript_id","transcript_pos"],right_on=["transcript_id","transcript_position"], how="left")

   return merged_data

  

In [3]:
data = parse_data("/Users/shaun/Desktop/DSA4266/project2/data.info",'/Users/shaun/Desktop/DSA4266/project2/dataset0.json.gz')

Loading /Users/shaun/Desktop/DSA4266/project2/dataset0.json.gz...
Loading /Users/shaun/Desktop/DSA4266/project2/data.info...
Transferring data from json to dataframe...
Merging dataframes to obtain labels


In [34]:
data

,transcript_id,transcript_pos,nucleo_seq,dwell_time_-1,sd_-1,mean_-1,dwell_time_0,sd_0,mean_0,dwell_time_1,sd_1,mean_1,gene_id,transcript_position,label
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,80.570270,ENSG00000004059,244,0
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,94.290698,ENSG00000004059,261,0
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,89.364324,ENSG00000004059,316,0
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006102,2.236520,89.154000,ENSG00000004059,332,0
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,85.178788,ENSG00000004059,368,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,4.005616,82.004110,ENSG00000167747,1348,1
121834,ENST00000641834,1429,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,3.644638,80.497101,ENSG00000167747,1429,0
121835,ENST00000641834,1531,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759687,113.562500,0.006410,2.181562,84.190625,ENSG00000167747,1531,1
121836,ENST00000641834,1537,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,2.540877,82.289474,ENSG00000167747,1537,0


Feature engineering 

One Hot Encoding of the sequences

In [4]:
def onehote(sequence):
    mapping = {"A": 0, "C": 1, "G": 2, "T": 3}
    seq2 = [mapping[i] for i in sequence]
    return np.eye(4)[seq2]

Get Kmer sequence

In [5]:
def kmers(sequence, ksize=5):
    kmers = []
    n_kmers = len(sequence) - ksize + 1

    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)

    return kmers

To get position weight matrix score for each nucleo_seq

In [12]:
def create_pwm_from_sequences(sequences):
    """
    Create a Position Weight Matrix (PWM) from a list of sequences.
    
    Args:
        sequences (list of str): List of DNA sequences of equal length.
        
    Returns:
        np.ndarray: The PWM matrix.
    """
    # Check if all sequences have the same length
    seq_length = len(sequences[0])
    if not all(len(seq) == seq_length for seq in sequences):
        raise ValueError("All sequences must have the same length")

    # Initialize the PWM matrix with zeros
    pwm = np.zeros((4, seq_length))  # 4 rows for A, C, G, T

    # Count the occurrences of each nucleotide at each position
    for seq in sequences:
        for i, nucleotide in enumerate(seq):
            if nucleotide == 'A':
                pwm[0, i] += 1
            elif nucleotide == 'C':
                pwm[1, i] += 1
            elif nucleotide == 'G':
                pwm[2, i] += 1
            elif nucleotide == 'T':
                pwm[3, i] += 1

    # Convert counts to probabilities
    pwm = pwm / len(sequences)

    return pd.DataFrame(data=pwm, columns=[1,2,3,4,5,6,7], index=["A","C","T","G"])

def log_odds(x):
    """
    Function to calculate log odds
    """
    if x == 0:
        return 0
    else:
        return np.log2(x/0.25)


def get_log_odds(sequences):
    """To get log odd dictionary which is used to calculate PWM score for each nucleo_seq"""
    ppm = create_pwm_from_sequences(sequences)
    log_odds_pos = ppm.applymap(log_odds)
    log_odds_dict = log_odds_pos.to_dict()

    return log_odds_dict

def get_PWM_score(seq):
    """Calculating PWM score for nucleo_seq based on the log odds"""
    res = 0
    for i in range(len(seq)):
        base = seq[i]
        dic = log_odds_dict[(i+1)]
        res = res + dic[base]
    return res

Data preprocessing

In [8]:
kmer_df = pd.DataFrame(data.apply(lambda x: kmers(x.nucleo_seq),axis=1).to_list(), columns=["nucleo_seq_-1","nucleo_seq_0","nucleo_seq_1"])
data = pd.concat([data,kmer_df], axis=1)

In [9]:
data["seq"] = data.apply(lambda x: onehote(x.nucleo_seq), axis=1)
data["seq_-1"] = data.apply(lambda x: onehote(x["nucleo_seq_-1"]), axis=1)
data["seq_0"] = data.apply(lambda x: onehote(x.nucleo_seq_0), axis=1)
data["seq_1"] = data.apply(lambda x: onehote(x.nucleo_seq_1), axis=1)


In [13]:
log_odds_dict = get_log_odds(data.nucleo_seq)


In [19]:
data["pwm_score"] = data.apply(lambda x: get_PWM_score(x.nucleo_seq),axis=1)

In [20]:
data

,transcript_id,transcript_pos,nucleo_seq,dwell_time_-1,sd_-1,mean_-1,dwell_time_0,sd_0,mean_0,dwell_time_1,...,transcript_position,label,nucleo_seq_-1,nucleo_seq_0,nucleo_seq_1,seq,seq_-1,seq_0,seq_1,pwm_score
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,244,0,AAGAC,AGACC,GACCA,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",5.265638
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,261,0,CAAAC,AAACT,AACTG,"[[0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",5.282447
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,316,0,GAAAC,AAACA,AACAG,"[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",6.220012
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006102,...,332,0,AGAAC,GAACA,AACAT,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",5.970300
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,368,0,AGGAC,GGACA,GACAA,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",5.146423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,1348,1,GGGAC,GGACA,GACAT,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",4.610474
121834,ENST00000641834,1429,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,1429,0,CTGAC,TGACA,GACAC,"[[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [...","[[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [...","[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",4.372075
121835,ENST00000641834,1531,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759687,113.562500,0.006410,...,1531,1,TGGAC,GGACA,GACAC,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",4.548100
121836,ENST00000641834,1537,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,1537,0,CTGAC,TGACC,GACCA,"[[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [...","[[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [...","[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [...","[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [...",4.355071


Train test split by gene ID

In [42]:
from sklearn.model_selection import GroupShuffleSplit 

splitter = GroupShuffleSplit(test_size=.10, n_splits=2, random_state = 42)
split = splitter.split(data, groups=data['gene_id'])
train_idx, test_idx = next(split)

train = data.iloc[train_idx]
test = data.iloc[test_idx]

In [45]:
train[train["gene_id"] == "ENSG00000006451"]

,transcript_id,transcript_pos,nucleo_seq,dwell_time_-1,sd_-1,mean_-1,dwell_time_0,sd_0,mean_0,dwell_time_1,...,gene_id,transcript_position,label,nucleo_seq_-1,nucleo_seq_0,nucleo_seq_1,seq,seq_-1,seq_0,seq_1
